In [31]:
# Install required libraries
!pip install pandas sentence-transformers

import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load the pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embedding(text):
    return model.encode(text)

# Load the data
df = pd.read_csv('../sample_files/emp.csv')

# Chunk the data
chunk_size = 100  # Adjust the chunk size as needed
chunks = [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]

# Create embeddings for each chunk
embeddings = []
for chunk in chunks:
    text = chunk.to_string(index=False)
    embedding = get_embedding(text)
    embeddings.append(embedding)

def chat_with_data(user_input):
    user_embedding = get_embedding(user_input)

    # Find the most similar chunk
    similarities = [util.cos_sim(user_embedding, emb).item() for emb in embeddings]
    most_similar_chunk_index = similarities.index(max(similarities))
    most_similar_chunk = chunks[most_similar_chunk_index]

    # Generate a response based on the most similar chunk
    response = generate_response(user_input)
    return response

def extract_empid(user_input):
    import re
    match = re.search(r'empid\s*=\s*(\d+)', user_input)
    if match:
        return int(match.group(1))
    return None

def extract_supervisorid(user_input):
    import re
    match = re.search(r'supervisorid\s*=\s*(\d+)', user_input)
    if match:
        return int(match.group(1))
    return None

def generate_response(user_input):
    if "how many records" in user_input.lower():
        total_records = df.shape[0]
        return f"There are {total_records} records in total."
    
    empid = extract_empid(user_input)
    if empid is not None:
        # Filter the entire dataset to get the specific employee's records
        emp_records = df[df['empid'] == empid]
        
        if emp_records.empty:
            return f"No records found for empid = {empid}"
        else:
            if "most recent pto" in user_input.lower():
                most_recent_record = emp_records.sort_values(by='payday', ascending=False).iloc[0]
                return f"The most recent PTO for empid = {empid} is {most_recent_record['pto']} hours."
            else:
                return f"Records for empid = {empid}:\n{emp_records.to_string(index=False)}"
    
    supervisorid = extract_supervisorid(user_input)
    if supervisorid is not None:
        # Filter the entire dataset to get the employees who have the specific supervisor
        managed_employees = df[df['supervisorid'] == supervisorid]
        
        if managed_employees.empty:
            return f"No employees found with supervisorid = {supervisorid}"
        else:
            return f"Employees with supervisorid = {supervisorid}:\n{managed_employees.to_string(index=False)}"
    
    return "I'm sorry, I didn't understand your question."

# Example usage
user_input = "how many records are there total"
response = chat_with_data(user_input)
print(response)

user_input = "Tell me about empid = 10"
response = chat_with_data(user_input)
print(response)

user_input = "what is the most recent pto for empid = 10"
response = chat_with_data(user_input)
print(response)

user_input = "what employees have supervisorid = 1"
response = chat_with_data(user_input)
print(response)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


There are 869 records in total.
Records for empid = 10:
 empid fname  lname  supervisorid  pto     payday  payamount
    10 Laura Wilson           3.0   40 2024-10-18     814.51
    10 Laura Wilson           3.0   48 2024-11-01     814.51
    10 Laura Wilson           3.0   56 2024-11-15     814.51
    10 Laura Wilson           3.0   64 2024-11-29     814.51
    10 Laura Wilson           3.0   72 2024-12-13     814.51
    10 Laura Wilson           3.0   40 2024-10-18     814.51
    10 Laura Wilson           3.0   48 2024-11-01     814.51
    10 Laura Wilson           3.0   56 2024-11-15     814.51
    10 Laura Wilson           3.0   64 2024-11-29     814.51
    10 Laura Wilson           3.0   72 2024-12-13     814.51
The most recent PTO for empid = 10 is 72 hours.
Employees with supervisorid = 1:
 empid   fname    lname  supervisorid  pto     payday  payamount
     2    Jane    Smith           1.0   24 2024-10-18      950.0
     2    Jane    Smith           1.0   32 2024-11-01      950

In [33]:
user_input = "what are the details of empid = 10"
response = chat_with_data("what is the most recent pto for empid = 10")
#response = chat_with_data("what is the last pay amount for empid = 10")
#response = chat_with_data("what employees have empid 1 as their supervisor")

print(response)

The most recent PTO for empid = 10 is 72 hours.
